In [2]:
import tensorflow as tf

First start with a model:

In [3]:
# Remove previous Tensors and Operations
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot=True)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


Let's train that model, then save the weights:

In [11]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 1000

saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})

        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.10660000145435333
Epoch 10  - Validation Accuracy: 0.2590000033378601
Epoch 20  - Validation Accuracy: 0.3937999904155731
Epoch 30  - Validation Accuracy: 0.4878000020980835
Epoch 40  - Validation Accuracy: 0.5577999949455261
Epoch 50  - Validation Accuracy: 0.6014000177383423
Epoch 60  - Validation Accuracy: 0.6349999904632568
Epoch 70  - Validation Accuracy: 0.6601999998092651
Epoch 80  - Validation Accuracy: 0.6830000281333923
Epoch 90  - Validation Accuracy: 0.7031999826431274
Epoch 100 - Validation Accuracy: 0.7218000292778015
Epoch 110 - Validation Accuracy: 0.7364000082015991
Epoch 120 - Validation Accuracy: 0.746399998664856
Epoch 130 - Validation Accuracy: 0.7573999762535095
Epoch 140 - Validation Accuracy: 0.7653999924659729
Epoch 150 - Validation Accuracy: 0.7706000208854675
Epoch 160 - Validation Accuracy: 0.776199996471405
Epoch 170 - Validation Accuracy: 0.7807999849319458
Epoch 180 - Validation Accuracy: 0.7870000004768372
Epoch 190 - V

Load a Trained Model
Let's load the weights and bias from memory, then check the test accuracy.

In [12]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.8712999820709229
